In [1]:
# services.py
# Autor: Pedro Casimiro
# Data: 26/11/2025

import pandas as pd
from src.config import PathConfig

class OcorrenciasService:
    """
    Camada de Serviço responsável por carregar e fornecer consultas
    ao DataFrame consolidado de ocorrências. O DF é carregado na memória
    apenas uma vez para garantir a performance da API.
    """
    
    # Variável de classe que armazenará o DataFrame (carregamento único)
    # Indica que a variável pertence à classe e não ao objeto
    df: pd.DataFrame = None

    @classmethod # Indica que o método é um método de classe
    def load_data(cls):
        """Carrega o DataFrame consolidado na memória, se ainda não estiver carregado."""
        if cls.df is None:
            print(f"[{__name__}] - Carregando DF consolidado de: {PathConfig.PATH_FINAL_DF.as_posix()}")
            try:
                # O método lê o arquivo que foi salvo pelo pipeline_etl.py
                cls.df = pd.read_csv(
                    PathConfig.PATH_FINAL_DF.as_posix(),
                    sep=';', 
                    encoding='utf-8')
                print(f"[{__name__}] - DF consolidado carregado com sucesso! Linhas: {len(cls.df)}")
            except FileNotFoundError:
                raise FileNotFoundError(
                    f"Arquivo de dados não encontrado em: {PathConfig.PATH_FINAL_DF.as_posix()}. "
                    "Execute 'python -m src.data.pipeline_etl' primeiro.")

    @classmethod
    def get_all_data(cls) -> list[dict]: # Indica que o método retorna uma lista de dicionários
        """
        Retorna todos os registros do DataFrame como uma lista de dicionários
        (formato JSON ideal para APIs).
        """
        # Garante que os dados estão carregados na memória
        if cls.df is None:
            cls.load_data()
            
        # Retorna o DF em memória convertido para uma lista de dicionários
        return cls.df.to_dict(orient='records')

    # Exemplo futuro: método para filtrar os dados
    @classmethod
    def filter_by_ra(cls, regiao_administrativa: str) -> list[dict]:
        """Filtra as ocorrências por Região Administrativa (case-insensitive)."""
        if cls.df is None:
            cls.load_data()

        # Cria uma máscara booleana para filtrar o DataFrame em memória
        mask = cls.df['RegiaoAdministrativa'].str.lower() == regiao_administrativa.lower()
        
        return cls.df[mask].to_dict(orient='records')

/tmp/ipykernel_93197/430925791.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# run_service_check.py
# Script de verificação manual para a classe OcorrenciasService

import json
from src.services.services import OcorrenciasService

# ----------------------------------------------------------------------
# Função principal de verificação
# ----------------------------------------------------------------------

def run_checks():
    """
    Carrega o DataFrame e exibe resultados de consultas de exemplo.
    """
    
    print("--- 1. INICIANDO VERIFICAÇÃO DO OCORRENCIAS SERVICE ---")
    
    # Chama o método que carrega o DF na memória. 
    # Se for a primeira execução, o carregamento real do CSV ocorrerá.
    OcorrenciasService.load_data()

    print("\n--- 2. VERIFICANDO PRIMEIRAS 5 LINHAS DO DF EM MEMÓRIA ---")
    
    # Acessa o DataFrame diretamente da variável de classe (para visualização)
    # e exibe as 5 primeiras linhas no formato tabular do Pandas.
    if OcorrenciasService.df is not None:
        print(OcorrenciasService.df.head().to_markdown(index=False))
        print(f"\nTotal de linhas carregadas: {len(OcorrenciasService.df)}")
    else:
        print("ERRO: DataFrame não foi carregado corretamente.")
        return

    # -------------------------------------------------------------
    
    print("\n--- 3. VERIFICANDO FORMATO DA API (get_all_data) ---")
    
    # Chama o método que retorna a lista de dicionários (formato JSON)
    all_data = OcorrenciasService.get_all_data()
    
    print(f"Número de registros: {len(all_data)}")
    print("Primeiro registro retornado (Formato API/JSON):")
    # Usa json.dumps para formatar o dicionário para melhor visualização no terminal
    print(json.dumps(all_data[0], indent=4, ensure_ascii=False))

    # -------------------------------------------------------------
    
    print("\n--- 4. TESTANDO FILTRO POR REGIÃO ADMINISTRATIVA ---")
    
    # Sugestão de RA para teste. Mude para uma RA que você sabe que tem dados.
    TEST_RA = "Plano Piloto"
    
    # Chama o método de filtro
    filtered_data = OcorrenciasService.filter_by_ra(TEST_RA)
    
    print(f"Consulta: Ocorrências em '{TEST_RA}'")
    print(f"Total de registros encontrados: {len(filtered_data)}")
    
    # Exibe o primeiro resultado filtrado
    if filtered_data:
        print("Primeiro registro filtrado:")
        print(json.dumps(filtered_data[0], indent=4, ensure_ascii=False))
    else:
        print(f"Atenção: Nenhum registro encontrado para '{TEST_RA}'. Tente outra RA.")

if __name__ == "__main__":
    run_checks()

--- 1. INICIANDO VERIFICAÇÃO DO OCORRENCIAS SERVICE ---

--- 2. VERIFICANDO PRIMEIRAS 5 LINHAS DO DF EM MEMÓRIA ---
|   id |   mes |   ano | RegiaoAdministrativa   | natureza   |   quantidade |
|-----:|------:|------:|:-----------------------|:-----------|-------------:|
|    1 |     1 |  2020 | ARNIQUEIRA             | ESTUPRO    |            1 |
|    2 |     2 |  2020 | ARNIQUEIRA             | ESTUPRO    |            2 |
|    3 |     3 |  2020 | ARNIQUEIRA             | ESTUPRO    |            0 |
|    4 |     4 |  2020 | ARNIQUEIRA             | ESTUPRO    |            2 |
|    5 |     5 |  2020 | ARNIQUEIRA             | ESTUPRO    |            0 |

Total de linhas carregadas: 33677

--- 3. VERIFICANDO FORMATO DA API (get_all_data) ---
Número de registros: 33677
Primeiro registro retornado (Formato API/JSON):
{
    "id": 1,
    "mes": 1,
    "ano": 2020,
    "RegiaoAdministrativa": "ARNIQUEIRA",
    "natureza": "ESTUPRO",
    "quantidade": 1
}

--- 4. TESTANDO FILTRO POR REGIÃO AD